In [1]:
#import basic libraries
import pandas as pd
import numpy as np
import random


#import SkLearn
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics.pairwise import cosine_similarity

import warnings
warnings.filterwarnings('ignore')


In [2]:
#import Instacart data

aisles = pd.read_csv('/Users/michaeljoyce/Desktop/Recommender_Engine_Instacart/data/aisles.csv')              
orders= pd.read_csv('/Users/michaeljoyce/Desktop/Recommender_Engine_Instacart/data/orders.csv') 
departments= pd.read_csv('/Users/michaeljoyce/Desktop/Recommender_Engine_Instacart/data/departments.csv')            
products= pd.read_csv('/Users/michaeljoyce/Desktop/Recommender_Engine_Instacart/data/products.csv') 
order_products_prior= pd.read_csv('/Users/michaeljoyce/Desktop/Recommender_Engine_Instacart/data/order_products__prior.csv')  
order_products_train= pd.read_csv('/Users/michaeljoyce/Desktop/Recommender_Engine_Instacart/data/order_products__train.csv') 

In [3]:
filenames=['aisles','departments', 'order_products__prior', 'order_products__train', 'orders', 'products']


In [4]:
#merge the datasets together

opt1=order_products_train.merge(products,how='left', on='product_id')
opt2=opt1.merge(departments,how='left', on='department_id')
opt3=opt2.merge(aisles, how='left', on='aisle_id')
opt=opt3.copy()

### Filter the data by reorders

In [5]:
# remove items that were not reordered

reorders=opt[opt['reordered']==1]

In [6]:
#filter for products reordered more than once 

hivol=reorders.copy()['product_id'].value_counts().sort_values(ascending=False)\
    [reorders.copy()['product_id'].value_counts().sort_values(ascending=False)>1].index.tolist()

reorders=reorders[reorders['product_id'].isin(hivol)]


In [7]:
#filter for high demand products

reorders['hi_dem']=(reorders.copy()['product_id'].value_counts().sort_values(ascending=False)>1)

hidem_ord = reorders[reorders['hi_dem'] == True]


In [8]:
#merge orders dataframe

user_orders=reorders.merge(orders)

In [9]:
#filter for high demand products

user_orders['hi_dem']=(user_orders.copy()['product_id'].value_counts().sort_values(ascending=False)>1)

hidem_ord=user_orders[user_orders['hi_dem']==True]

### Create the recommender system

In [10]:
def Recommender_System(user_id):
    
    '''
    Input User Id to see recommendations for the User
    '''
    
    u = hidem_ord.groupby(['user_id','product_name']).size().sort_values(ascending=False).unstack().fillna(0)
    users_sim = pd.DataFrame(cosine_similarity(u),index=u.index,columns=u.index)

    p = hidem_ord.groupby(['product_name','user_id']).size().sort_values(ascending=False).unstack().fillna(0)
    
    recommendations = pd.Series(np.dot(p.values,users_sim[user_id]), index=p.index)
    return recommendations.sort_values(ascending=False).head(),

In [11]:
#gives a random user ID to for input to the recommender system
example=random.sample(hidem_ord['user_id'].tolist(),1)[0]
print(example)

139903


In [12]:
Recommender_System(example)

(product_name
 Spinaci e Ricotta Ravioli                              1.267261
 Smmmile, It's Three Cheesy Macaroni & Cheese Dinner    1.000000
 Original No Pulp 100% Florida Orange Juice             0.267261
 Deli Fresh Shaved Smoked Turkey Breast                 0.267261
 Family Size Lasagna With Meat & Sauce                  0.267261
 dtype: float64,)